In [17]:
import tensorflow as tf

tf.reset_default_graph()
# with tf.device("/job:ps"):
c = tf.constant(15.0)

# with tf.device("/job:worker"):
a = c+5

b = c*a

## shared variable in the cluster
x = tf.Variable(0.0, name="x")
increment_x = tf.assign(x, x + 1)

with tf.Session("grpc://localhost:2221") as sess:
    sess.run(x.initializer)
    sess.run(increment_x)
    print(b.eval(), x.eval())

300.0 1.0


In [6]:
import tensorflow as tf

tf.reset_default_graph()
with tf.device(tf.train.replica_device_setter(ps_tasks=1)):
    var=[]
    for i in range(5):
        var.append(tf.Variable(float(i+1)))

    s=var[0]+var[1]

    with tf.device("/job:ps"):
        p1 = 2*s
        tf.constant(10, name="shared_constant")
        init = tf.global_variables_initializer()

with tf.Session("grpc://localhost:2221") as sess:
    sess.run(init)
    r = sess.run([s,p1])
    print(r)
                   

[3.0, 6.0]


<tf.Tensor 'shared_constant:0' shape=() dtype=int32>